# Single-task Test-time fine-tuning for ARC25 (nikimakarov)

## 0. Configuration and imports

In [1]:
import sys
import os

DEBUG = True  # Если нужно, можно отключить, чтобы код не выполнялся

if DEBUG:
    # Add the arc24-source-code directory to Python path
    arc24_path = os.path.abspath('kaggle/input/arc24-source-code')
    if arc24_path not in sys.path:
        sys.path.append(arc24_path)

    # Import the TTFTDatasetCreator directly from the module file
    from arc25.create_test_dataset import TTFTDatasetCreator

    # Создаем тестовые датасеты
    creator = TTFTDatasetCreator()
    original, n_minus_1 = creator.create_ttft_test_datasets(
        input_path='kaggle/input/arc-prize-2025/arc-agi_training_challenges.json',
        num_tasks=10
    )

�� Создание тестовых датасетов для TTFT...
✅ Загружен датасет из kaggle/input/arc-prize-2025/arc-agi_training_challenges.json: 1000 задач
📊 Найдено 1000 валидных задач из 1000
💾 Датасет сохранен: ./small_test_dataset.json
✅ Создан датасет с 10 задачами:
  📋 aa18de87: 4 train, 1 test
  📋 22233c11: 3 train, 1 test
  📋 0b17323b: 2 train, 1 test
  📋 c4d1a9ae: 2 train, 1 test
  📋 4cd1b7b2: 3 train, 1 test
  📋 44f52bb0: 6 train, 2 test
  📋 3f23242b: 2 train, 1 test
  📋 278e5215: 3 train, 1 test
  📋 c35c1b4c: 3 train, 1 test
  📋 2037f2c7: 3 train, 1 test
✅ Создан N-1 датасет с 10 задачами
💾 Датасет сохранен: ./small_test_dataset_n-1.json

📁 Файлы сохранены в .:
  - small_test_dataset.json (оригинальный датасет)
  - small_test_dataset_n-1.json (N-1 датасет для TTFT)


In [2]:
#qwen25-0.5b/8 100split step32k_bs1 8e-5lr_lin 96E
n_splits = 100 #2, 4, 10, 20, 50, 100
total_train_steps = 32000
class cfg:
    # Model
    model_path = '/kaggle/input/qwen2.5/transformers/0.5b-instruct/1'
    input_lora_path = '/kaggle/input/loras/transformers/qwen2.5-0.5b-instruct/8'
    prompt_version = 'output-from-examples-v1'
    merged_model_path = '/kaggle/tmp/qwen_merged_model'
    grid_encoder = 'GridShapeEncoder(RowNumberEncoder(MinimalGridEncoder()))' #GridCodeBlockEncoder(MinimalGridEncoder())
    max_model_len = 10240
    # Dataset
    if DEBUG:
        dataset_path = 'small_test_dataset_n-1.json'
    else:
        dataset_path = '/kaggle/input/arc-prize-2025/arc-agi_test_challenges.json'
    #dataset_path = '/kaggle/input/arc24-source-code/new_partitions/val_rs7.json'
    #dataset_path = 'smaller_val_challenges.json'
    split_size = 100//n_splits # How many tasks there would be on each split, use 1 for the canonical single-task test-time fine-tuning
    # Fine-tuning params
    max_steps = total_train_steps//n_splits
    learning_rate = 8e-5 #1e-4 for lora smaller than 512
    lr_scheduler_type: str = "linear" #linear, constant_with_warmup, cosine, cosine_with_restarts
    batch_size = 1
    max_seq_len = 5120 #3456, 5120
    # Inference params
    predictions_per_task = 96 # multiple of 8
    inference_timeout = "12m" # max inference time per split, I estimate that for 128 preds Qwen-0.5B takes 6 min in the worst case (50 splits)
    # Ensemble
    ensemble_with_2020: bool = True

import os
is_dry_run = cfg.dataset_path == '/kaggle/input/arc-prize-2025/arc-agi_test_challenges.json' and not os.getenv('KAGGLE_IS_COMPETITION_RERUN')
if is_dry_run:
    print('This is a dry run, no inference nor installation of packages will be done')
    
if int(cfg.input_lora_path.split('/')[-1]) < 18 and cfg.input_lora_path.startswith('/kaggle/input/loras/transformers/qwen2-0.5b'):
    assert cfg.prompt_version == 'output-from-examples-v0'
else:
    assert cfg.prompt_version == 'output-from-examples-v1'

In [4]:
pip install tqdm

  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
Note: you may need to restart the kernel to use updated packages.


In [5]:
import logging
import subprocess
import sys
import json
import glob
import os
import shutil
from tqdm.auto import tqdm

if not is_dry_run:
    sys.path.append('kaggle/input/arc24-source-code')

# Configure logging to output to the notebook console
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    force=True
)

/Users/nikitamakarov/Desktop/ARC_RELOAD/.conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 1. Libraries and resource monitor

In [9]:
%%time
if DEBUG:
    # Удаляем torch и ставим локальные библиотеки
    !bash ./install_libraries_local.sh
else:
    # Если не dry-run, устанавливаем библиотеки из kaggle/input
    if not is_dry_run:
        !bash kaggle/input/arc24-source-code/install_libraries.sh

🔧 Installing libraries from wheels...
�� Installing vllm and dependencies...
Looking in links: kaggle/input/making-wheels-of-necessary-packages-for-vllm
ERROR: Could not find a version that satisfies the requirement vllm (from versions: none)
ERROR: No matching distribution found for vllm
Looking in links: kaggle/input/making-wheels-of-necessary-packages-for-vllm
Processing ./kaggle/input/making-wheels-of-necessary-packages-for-vllm/peft-0.12.0-py3-none-any.whl
INFO: pip is looking at multiple versions of peft to determine which version is compatible with other requirements. This could take a while.
ERROR: Could not find a version that satisfies the requirement torch>=1.13.0 (from peft) (from versions: none)
ERROR: No matching distribution found for torch>=1.13.0
Looking in links: kaggle/input/making-wheels-of-necessary-packages-for-vllm
Processing ./kaggle/input/making-wheels-of-necessary-packages-for-vllm/trl-0.9.6.dev0-py3-none-any.whl
INFO: pip is looking at multiple versions of tr

In [5]:
!pip install torch
!pip install trl

In [11]:
pip install matplotlib

  Using cached contourpy-1.3.2-cp310-cp310-macosx_11_0_arm64.whl.metadata (5.5 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 19.6 MB/s  0:00:00 eta 0:00:01
Using cached contourpy-1.3.2-cp310-cp310-macosx_11_0_arm64.whl (253 kB)
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 16.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6/6 [matplotlib]6 [matplotlib]
Note: you may need to restart the kernel to use updated packages.


In [12]:
if not is_dry_run:    
    from arc24.utils import ResourceMonitor    
    monitor = ResourceMonitor(interval=1)    
    monitor.start()

## 2. Prepare data for training

In [13]:
if not is_dry_run:
    single_task_datasets_path = 'single_task_datasets'
    os.makedirs(single_task_datasets_path, exist_ok=True)

    # Читаем датасет
    with open(cfg.dataset_path, 'r') as f:
        items = list(json.load(f).items())
        assert len(items) % cfg.split_size == 0

        print(len(items))

        # Создаем датасеты по одному таску
        for batch_idx in tqdm(range(len(items) // cfg.split_size), desc='Creating single task datasets'):
            data = dict(items[batch_idx * cfg.split_size: (batch_idx + 1) * cfg.split_size])
            assert len(data) == cfg.split_size

            task_id = list(data.keys())[0]
            with open(os.path.join(single_task_datasets_path, f'{task_id}.json'), 'w') as out_f:
                json.dump(data, out_f)

10


Creating single task datasets: 100%|██████████| 10/10 [00:00<00:00, 3628.92it/s]


In [14]:
! ls -lh {single_task_datasets_path}

total 88
-rw-r--r--@ 1 nikitamakarov  staff   2.8K Sep 26 08:23 0b17323b.json
-rw-r--r--@ 1 nikitamakarov  staff   5.4K Sep 26 08:23 2037f2c7.json
-rw-r--r--@ 1 nikitamakarov  staff   2.0K Sep 26 08:23 22233c11.json
-rw-r--r--@ 1 nikitamakarov  staff   2.4K Sep 26 08:23 278e5215.json
-rw-r--r--@ 1 nikitamakarov  staff   2.3K Sep 26 08:23 3f23242b.json
-rw-r--r--@ 1 nikitamakarov  staff   413B Sep 26 08:23 44f52bb0.json
-rw-r--r--@ 1 nikitamakarov  staff   446B Sep 26 08:23 4cd1b7b2.json
-rw-r--r--@ 1 nikitamakarov  staff   1.1K Sep 26 08:23 aa18de87.json
-rw-r--r--@ 1 nikitamakarov  staff   2.0K Sep 26 08:23 c35c1b4c.json
-rw-r--r--@ 1 nikitamakarov  staff   1.1K Sep 26 08:23 c4d1a9ae.json


In [15]:
if not is_dry_run:
    training_datasets_path = 'single_task_training_datasets'
    os.makedirs(training_datasets_path, exist_ok=True)

    dataset_filepaths = glob.glob(os.path.join(single_task_datasets_path, '*.json'))

    # Создаем n-1 датасеты для обучения
    for dataset_filepath in tqdm(dataset_filepaths, desc='Creating ttft training datasets'):
        !python kaggle/input/arc24-source-code/create_n-1_dataset.py \
            {dataset_filepath} \
            {os.path.join(training_datasets_path, os.path.basename(dataset_filepath))}

Creating ttft training datasets:   0%|          | 0/10 [00:00<?, ?it/s]

Namespace(input_dataset='single_task_datasets/44f52bb0.json', output_dataset='single_task_training_datasets/44f52bb0.json')
single_task_datasets/44f52bb0.json has 1 tasks
New n-1 dataset has 1 tasks


Creating ttft training datasets:  10%|█         | 1/10 [00:00<00:02,  4.08it/s]

Namespace(input_dataset='single_task_datasets/2037f2c7.json', output_dataset='single_task_training_datasets/2037f2c7.json')
single_task_datasets/2037f2c7.json has 1 tasks
New n-1 dataset has 1 tasks


Creating ttft training datasets:  20%|██        | 2/10 [00:00<00:01,  4.13it/s]

...

Creating ttft training datasets:  80%|████████  | 8/10 [00:01<00:00,  4.13it/s]

Namespace(input_dataset='single_task_datasets/3f23242b.json', output_dataset='single_task_training_datasets/3f23242b.json')
single_task_datasets/3f23242b.json has 1 tasks
New n-1 dataset has 0 tasks


Creating ttft training datasets:  90%|█████████ | 9/10 [00:02<00:00,  4.14it/s]

Namespace(input_dataset='single_task_datasets/4cd1b7b2.json', output_dataset='single_task_training_datasets/4cd1b7b2.json')
single_task_datasets/4cd1b7b2.json has 1 tasks
New n-1 dataset has 1 tasks


Creating ttft training datasets: 100%|██████████| 10/10 [00:02<00:00,  4.12it/s]


## 3. Test-time fine-tuning

In [16]:
%%time
def clean_train_output_except_adapter(output_dir):
    """
    Max Disk is 57.6GiB, but around 8GiB are already used.
    If each checkpoint weights 265M, that would be 26GB for 100 savings. 
    The optimizer was using just 133M.

    However, on '/kaggle/working/' we can only save 20GB. 
    The run that had the disk error used 50 splits, 
    each split was using around 400M, so it makes sense.
    """

    # Удаляем лишние файлы, оставляя только адаптеры
    !rm "{output_dir}"/*/*.pth "{output_dir}"/*/*.pt "{output_dir}"/*/*.md "{output_dir}"/*/*.txt "{output_dir}"/*/*.bin "{output_dir}"/*/token*
    !rm "{output_dir}"/*/added_tokens.json "{output_dir}"/*/special_tokens_map.json "{output_dir}"/*/vocab.json "{output_dir}"/*/trainer_state.json


if not is_dry_run:
    dataset_filepaths = sorted(glob.glob(os.path.join(training_datasets_path, '*.json')))
    checkpoints_folder = '.tmp/checkpoints'  # https://www.kaggle.com/docs/notebooks#technical-specifications
    os.makedirs(checkpoints_folder, exist_ok=True)

    # Запускаем fine-tuning для каждого датасета
    for dataset_filepath in tqdm(dataset_filepaths, desc='Finetuning models'):
        output_dir = os.path.join(checkpoints_folder, os.path.splitext(os.path.basename(dataset_filepath))[0])

        !python kaggle/input/arc24-source-code/fine-tuning.py \
            --model_path={cfg.model_path} \
            --adapter_path={cfg.input_lora_path} \
            --output_dir={output_dir} \
            --train_datasets {dataset_filepath} {cfg.prompt_version} \
            --val_dataset {dataset_filepath} {cfg.prompt_version} \
            --max_steps={cfg.max_steps} \
            --eval_steps={cfg.max_steps*2} \
            --max_seq_len={cfg.max_seq_len} \
            --learning_rate={cfg.learning_rate} \
            --lr_scheduler_type={cfg.lr_scheduler_type} \
            --batch_size={cfg.batch_size} \
            --report_to=tensorboard \
            --grid_encoder="{cfg.grid_encoder}" \
            --remove_train_samples_to_fit_max_seq_len \
            --torch_dtype=float16 \
            --no-verbose

        # Очищаем лишние файлы после каждого запуска
        clean_train_output_except_adapter(output_dir)

        logging.info(f'Finished fine-tuning for split {dataset_filepaths.index(dataset_filepath) + 1}/{len(dataset_filepaths)}')

Finetuning models:   0%|          | 0/10 [00:00<?, ?it/s]

Traceback (most recent call last):
  File "/Users/nikitamakarov/Desktop/ARC_RELOAD/kaggle/input/arc24-source-code/fine-tuning.py", line 15, in <module>
    import torch
ModuleNotFoundError: No module named 'torch'
zsh:1: no matches found: .tmp/checkpoints/0b17323b/*/*.pth
zsh:1: no matches found: .tmp/checkpoints/0b17323b/*/added_tokens.json


2025-09-26 08:23:45,575 - INFO - Finished fine-tuning for split 1/10
Finetuning models:  10%|█         | 1/10 [00:01<00:17,  1.92s/it]

Traceback (most recent call last):
  File "/Users/nikitamakarov/Desktop/ARC_RELOAD/kaggle/input/arc24-source-code/fine-tuning.py", line 15, in <module>
    import torch
ModuleNotFoundError: No module named 'torch'
zsh:1: no matches found: .tmp/checkpoints/2037f2c7/*/*.pth
zsh:1: no matches found: .tmp/checkpoints/2037f2c7/*/added_tokens.json


2025-09-26 08:23:46,699 - INFO - Finished fine-tuning for split 2/10
Finetuning models:  20%|██        | 2/10 [00:03<00:11,  1.45s/it]

...

Traceback (most recent call last):
  File "/Users/nikitamakarov/Desktop/ARC_RELOAD/kaggle/input/arc24-source-code/fine-tuning.py", line 15, in <module>
    import torch
ModuleNotFoundError: No module named 'torch'
zsh:1: no matches found: .tmp/checkpoints/c35c1b4c/*/*.pth
zsh:1: no matches found: .tmp/checkpoints/c35c1b4c/*/added_tokens.json


2025-09-26 08:23:54,580 - INFO - Finished fine-tuning for split 9/10
Finetuning models:  90%|█████████ | 9/10 [00:10<00:01,  1.14s/it]

Traceback (most recent call last):
  File "/Users/nikitamakarov/Desktop/ARC_RELOAD/kaggle/input/arc24-source-code/fine-tuning.py", line 15, in <module>
    import torch
ModuleNotFoundError: No module named 'torch'
zsh:1: no matches found: .tmp/checkpoints/c4d1a9ae/*/*.pth
zsh:1: no matches found: .tmp/checkpoints/c4d1a9ae/*/added_tokens.json


2025-09-26 08:23:55,688 - INFO - Finished fine-tuning for split 10/10
Finetuning models: 100%|██████████| 10/10 [00:12<00:00,  1.20s/it]

CPU times: user 63.6 ms, sys: 151 ms, total: 215 ms
Wall time: 12 s


In [ ]:
!ls -lh {checkpoints_folder}/*/checkpoint*/adapter_model.safetensors